In [58]:
import torch
import pandas as pd
from tqdm import tqdm
import time

In [59]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [60]:
# Будем использовать для обучения датасет с репликами из "Симпсонов"
df = pd.read_csv(r"D:\НЕТОЛОГИЯ\Deep ML\файлы\simpsons_script_lines.csv")
df.head()

C:\Users\innok\AppData\Local\Temp\ipykernel_2324\1709974256.py:2: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"D:\НЕТОЛОГИЯ\Deep ML\файлы\simpsons_script_lines.csv")


,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,True,464.0,3.0,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31
1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,True,9.0,3.0,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,wheres mr bergstrom,3
2,9551,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,True,464.0,3.0,Miss Hoover,Springfield Elementary School,I don't know. Although I'd sure like to talk t...,i dont know although id sure like to talk to h...,22
3,9552,32,212,Lisa Simpson: That life is worth living.,864000,True,9.0,3.0,Lisa Simpson,Springfield Elementary School,That life is worth living.,that life is worth living,5
4,9553,32,213,Edna Krabappel-Flanders: The polls will be ope...,864000,True,40.0,3.0,Edna Krabappel-Flanders,Springfield Elementary School,The polls will be open from now until the end ...,the polls will be open from now until the end ...,33


### Задание 1.

**1.1. Алгоритм шифра Цезаря**

In [61]:
# Фукция для шифра Цезаря
def ceaser_code(text, alphabet, step):
    new_text = ''
    for sym in text:
        if sym in alphabet:
            sym_ind = alphabet.index(sym)
            if sym_ind >= len(alphabet)-step:
               new_ind = alphabet.index(sym) - ((len(alphabet)-1)*2)
            else: 
                new_ind = alphabet.index(sym) + step 
            new_sym = alphabet[new_ind]
            new_text += new_sym
            
        else: new_text += sym
    return new_text

In [62]:
# Демонтстрация работы шифра
alphabet = 'abcdefghijklmnopqrstuvwxyz'
phrase = df['normalized_text'][1]

print('исходня строка:', phrase)
print('зашифрованная строка:', ceaser_code(phrase, alphabet, 2))

исходня строка: wheres mr bergstrom
зашифрованная строка: yjgtgu ot dgtiuvtqo


**1.2. Кодировка данных**

In [63]:
# Даные для модели. Исходный текст и зашифрованный вариант
phrases = df['normalized_text'].tolist()
text = [phrase for phrase in phrases if type(phrase) is str]
text_cypher = [ceaser_code(phrase, alphabet, 2) for phrase in text]

# Разделим слова в полученных массивах на символы
text = [[c for c in ph] for ph in text]
text_cypher = [[c for c in ph] for ph in text_cypher]

In [64]:
# Кодировка сомволов числами
CHARS = set('abcdefghijklmnopqrstuvwxyz ')  # все символы, которые мы хотим использовать для кодировки = наш словарь
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]  # все неизвестные символы будут получать тег none
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}  # словарь токен-индекс

In [65]:
MAX_LEN = 50  # мы хотим ограничить максимальную длину ввода

X = torch.zeros((len(text), MAX_LEN), dtype=int)  # создаём пустой вектор для текста, чтобы класть в него индексы токенов

for i in range(len(text)):  # для каждого предложения
    for j, w in enumerate(text[i]):  # для каждого токена
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [66]:
# То же для зашифорованных текстов
X_cypher = torch.zeros((len(text_cypher), MAX_LEN), dtype=int)  

for i in range(len(text_cypher)):  
    for j, w in enumerate(text_cypher[i]): 
        if j >= MAX_LEN:
            break
        X_cypher[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

**1.3. RNN**

In [67]:
# Задаем сеть
class RNN(torch.nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.embedding = torch.nn.Embedding(28, 30)
        self.rnn = torch.nn.RNN(30, 128)
        self.out = torch.nn.Linear(128, 28)

    def forward(self, sentences, state=None):
        x = self.embedding(sentences)
        x, s = self.rnn(x) # берём выход с последнего слоя для всех токенов, а не скрытое состояние
        return self.out(x)
        
model = RNN()
model = model.to(device)

In [85]:
#Лосс и оптимизатор
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [69]:
# Обучение. На вход - зашифрованные тексты (X_cypher), на выход - исходные (X)
for ep in tqdm(range(20)):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        # берём батч в 100 элементов
        X_cypher_batch = X_cypher[i * 100:(i + 1) * 100]
        X_batch = X[i * 100:(i + 1) * 100].flatten()

        X_cypher_batch = X_cypher_batch.to(device)
        X_batch = X_batch.to(device)

        optimizer.zero_grad()
        answers = model.forward(X_cypher_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, X_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

  5%|▌         | 1/20 [00:02<00:53,  2.82s/it]

Epoch 0. Time: 2.824, Train loss: 0.786


 10%|█         | 2/20 [00:05<00:48,  2.71s/it]

Epoch 1. Time: 2.633, Train loss: 0.118


 15%|█▌        | 3/20 [00:08<00:45,  2.68s/it]

...

Epoch 17. Time: 2.745, Train loss: 0.005


 95%|█████████▌| 19/20 [00:50<00:02,  2.65s/it]

Epoch 18. Time: 2.558, Train loss: 0.005


100%|██████████| 20/20 [00:53<00:00,  2.67s/it]

Epoch 19. Time: 2.542, Train loss: 0.004


**1.4. Проверка работы сети**

In [77]:
# Функция для генерации дешифрованной последовательности
def net_decypher(word):
    sentence = list(word)
    sentence = [CHAR_TO_INDEX.get(s, 0) for s in sentence]
    answers = model.to('cpu').forward(torch.tensor(sentence))
    probas, indices = answers.topk(1)
    return ''.join([INDEX_TO_CHAR[ind.item()] for ind in indices.flatten()])

In [71]:
# Зашифруем тексты
text_long = 'pytorch is an optimized tensor library for deep learning using gpus and cpus' 
text_short = 'recurrent neural networks' 

cypher_long = ceaser_code(text_long, alphabet, 2)
cypher_short = ceaser_code(text_short, alphabet, 2)

print(cypher_long)
print(cypher_short)

ravqtej ku cp qrvkokbgf vgpuqt nkdtcta hqt fggr ngctpkpi wukpi irwu cpf erwu
tgewttgpv pgwtcn pgvyqtmu


In [78]:
# Расшифруем тексты с помощью сети
print(net_decypher(cypher_long))
print(net_decypher(cypher_short))

pytorch is an optimized tensor library for deep learning using gpus and cpus
recurrent neural networks


### Задание 2.

In [84]:
# Новая модель на основе полносвязных слоев
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.embedding = torch.nn.Embedding(28, 30)
        self.fc1 = torch.nn.Linear(30, 256)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(256, 128)
        self.relu = torch.nn.ReLU()
        self.out = torch.nn.Linear(128, 28)

    def forward(self, sentences, state=None):
        x = self.embedding(sentences)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.out(x)
        return x
        
model_mlp = MLP()
model_mlp = model_mlp.to(device)

optimizer_mlp = torch.optim.SGD(model_mlp.parameters(), lr = 0.01)

In [87]:
# Обучение новой сети
for ep in tqdm(range(20)):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        # берём батч в 100 элементов
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()
        
        X_batch = X_batch.to(device)
        Y_batch = Y_batch.to(device)
        
        optimizer_mlp.zero_grad()
        answers = model_mlp.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer_mlp.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

  5%|▌         | 1/20 [00:03<01:02,  3.26s/it]

Epoch 0. Time: 3.263, Train loss: 2.030


 10%|█         | 2/20 [00:06<00:56,  3.13s/it]

Epoch 1. Time: 3.036, Train loss: 1.812


 15%|█▌        | 3/20 [00:09<00:52,  3.10s/it]

...

Epoch 17. Time: 3.115, Train loss: 1.694


 95%|█████████▌| 19/20 [00:58<00:03,  3.11s/it]

Epoch 18. Time: 3.103, Train loss: 1.693


100%|██████████| 20/20 [01:01<00:00,  3.10s/it]

Epoch 19. Time: 3.076, Train loss: 1.692


In [90]:
# Генерация текста новой моделью
def generate_sentence(word):
    sentence = list(word)
    sentence = [CHAR_TO_INDEX.get(s, 0) for s in sentence]
    answers = model_mlp.to('cpu').forward(torch.tensor(sentence))
    probas, indices = answers.topk(1)
    return ''.join([INDEX_TO_CHAR[ind.item()] for ind in indices.flatten()])

In [94]:
generate_sentence('simpsons')

' nee u  '